In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_stata('Dataset_merged.dta').drop(columns=['timestamp', '_merge']).dropna()
df['userid'] = df['userid'].astype(int)
df.dtypes
df.head()

,userid,movieid,rating,title
0,55889,1,3.5,Toy Story (1995)
1,64965,1,4.5,Toy Story (1995)
2,61730,1,3.0,Toy Story (1995)
3,68941,1,5.0,Toy Story (1995)
4,34902,1,5.0,Toy Story (1995)


In [3]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=0)
#train
df_movie_features1 = df_train.pivot(
    index='movieid',
    columns='userid',
    values='rating'
).fillna(0)
#Test
df_movie_features2 = df_test.pivot(
    index='movieid',
    columns='userid',
    values='rating'
).fillna(0)


In [ ]:
#Standardizing
#Train
piv_norm_train = df_movie_features1.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# Drop all columns containing only zeros representing users who did not rate
piv_norm_train.fillna(0, inplace=True)
piv_norm_train = piv_norm_train.T
piv_norm_train = piv_norm_train.loc[:, (piv_norm_train != 0).any(axis=0)]

In [ ]:
#Standardizing
#Test
piv_norm_test = df_movie_features2.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# Drop all columns containing only zeros representing users who did not rate
piv_norm_test.fillna(0, inplace=True)
piv_norm_test = piv_norm_test.T
piv_norm_test = piv_norm_test.loc[:, (piv_norm_test != 0).any(axis=0)]

In [ ]:
#Sparse Matrices
piv_train_sparse = sp.sparse.csr_matrix(piv_norm_train.values)
piv_test_sparse = sp.sparse.csr_matrix(piv_norm_test.values)

In [ ]:
#Calculating Similarities
#Train
user_similarity_train = cosine_similarity(piv_train_sparse)
item_similarity_train = cosine_similarity(piv_train_sparse.T)

#Test
user_similarity_test = cosine_similarity(piv_test_sparse)
item_similarity_test = cosine_similarity(piv_test_sparse.T)

In [ ]:
#Train
item_train_sim_df = pd.DataFrame(item_similarity_train, index = piv_norm_train.index, columns = piv_norm_train.index)
user_train_sim_df = pd.DataFrame(user_similarity_train, index = piv_norm_train.columns, columns = piv_norm_train.columns)

#Test
item_test_sim_df = pd.DataFrame(item_similarity_test, index = piv_norm_test.index, columns = piv_norm_test.index)
user_test_sim_df = pd.DataFrame(user_similarity_test, index = piv_norm_test.columns, columns = piv_norm_test.columns)